## Data Preprocessing

In [63]:
# Download "Residential Building" and "Real Property Sales" from King County Website
# http://info.kingcounty.gov/assessor/DataDownload/default.aspx
# Put in the same folder as this file
# Output will be two new data files: 
# 1) "Renovated.csv" - Residential buildings that have been renovated (ParcelId as the unique identifier)
# 2) "MultiSale.csv" - Residential buildings that have been renovated and sold more than once

In [64]:
import pandas as pd

# I. import residential data
residential_dtype = {"Major": str, "Minor": str}
residential_df = pd.read_csv("EXTR_ResBldg.csv", dtype = residential_dtype)

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [65]:
residential_df.head(8)

,Major,Minor,BldgNbr,NbrLivingUnits,Address,BuildingNumber,Fraction,DirectionPrefix,StreetName,StreetType,...,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
0,034570,0100,1,1,2101 NE 23RD ST 98056,2101,,NE,23RD,ST,...,0,0,0,1981,0,0,0,0,3,0
1,034570,0140,1,1,2207 NE 23RD ST 98056,2207,,NE,23RD,ST,...,0,1,0,1981,0,0,0,0,3,0
2,034570,0350,1,1,2320 BLAINE AVE NE 98056,2320,,,BLAINE,AVE,...,0,0,0,1981,2003,0,0,0,3,3500
3,034800,0090,1,1,523 S 17TH ST 98055,523,,S,17TH,ST,...,1,0,1,1967,0,0,0,0,3,0
4,034800,0095,1,1,517 S 17TH ST 98055,517,,S,17TH,ST,...,1,0,1,1967,0,0,0,0,3,0
5,034800,0100,1,1,1716 MORRIS AVE S 98055,1716,,,MORRIS,AVE,...,1,0,1,1967,0,0,0,0,4,0
6,034800,0105,1,1,1722 MORRIS AVE S 98055,1722,,,MORRIS,AVE,...,1,0,1,1969,0,0,0,0,3,0
7,034940,0140,1,1,2243 SCANDIA AVE 98022,2243,,,SCANDIA,AVE,...,0,0,0,1980,0,0,0,0,4,0


In [66]:
residential_df.iloc[0]

Major                                    034570
Minor                                      0100
BldgNbr                                       1
NbrLivingUnits                                1
Address                2101  NE 23RD ST   98056
BuildingNumber                            2101 
Fraction                                       
DirectionPrefix                              NE
StreetName            23RD                     
StreetType                               ST    
DirectionSuffix                                
ZipCode                                   98056
Stories                                       1
BldgGrade                                     7
BldgGradeVar                                  0
SqFt1stFloor                                790
SqFtHalfFloor                                 0
SqFt2ndFloor                                  0
SqFtUpperFloor                                0
SqFtUnfinFull                                 0
SqFtUnfinHalf                           

In [67]:
# Check missing values
na = (residential_df.isnull().sum() / len(residential_df)) * 100
na = na.drop(na[na == 0].index).sort_values(ascending=False)
na_df = pd.DataFrame({'Missing Ratio' :na})
na_df

,Missing Ratio
ZipCode,7.938753
DirectionSuffix,0.108570
DirectionPrefix,0.108570


In [68]:
residential_df.YrRenovated.value_counts().head()
sum(residential_df.YrRenovated.value_counts()[1:])

23150

In [69]:
# keep only houses that have been renovated
renovated_df = residential_df[residential_df.YrRenovated != 0].copy()
renovated_df.shape

(23150, 50)

In [70]:
# add ParcelId column and check uniqueness
renovated_df["ParcelId"] = renovated_df["Major"]+renovated_df["Minor"]
renovated_df.head()
len(renovated_df.ParcelId.unique())
renovated_df.ParcelId.value_counts().sort_values()

0221029094    1
2223059041    1
5216200200    1
9274202595    1
5141000720    1
8856004415    1
1025039253    1
2632000110    1
3362900656    1
2883201380    1
0809000635    1
2128000210    1
9209900535    1
1643500165    1
0856000330    1
1320069238    1
3623079043    1
4363700390    1
2131701725    1
1234000540    1
2206500435    1
7579200560    1
7011200865    1
6189200380    1
3121069044    1
3271800030    1
9274201820    1
7116000715    1
7518504280    1
3625059064    1
             ..
2523069111    2
0449000060    2
9187200300    2
1824059033    2
3362900760    2
3223039026    2
3451000043    2
3761100055    2
1422059030    2
6610000525    2
4197400060    2
1257203044    2
6199000262    2
3426049165    2
4014400080    2
7547300920    2
0221029021    2
1322059056    2
2623029004    2
1422039011    2
2433200045    2
0617000095    2
1525059122    2
7204200025    2
1024079026    2
2979800195    2
1023089049    3
0922059117    3
3822200041    4
5119400095    7
Name: ParcelId, Length: 

In [71]:
renovated_df[renovated_df.ParcelId == "5119400095"]
# One parcel ID can serve multiple buildings. 
# Delete all parcel IDs with multiple units.

,Major,Minor,BldgNbr,NbrLivingUnits,Address,BuildingNumber,Fraction,DirectionPrefix,StreetName,StreetType,...,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost,ParcelId
294782,511940,0095,1,1,23641 7TH AVE S 98198,23641,,,7TH,AVE,...,1,0,1930,1961,0,0,0,4,0,5119400095
294784,511940,0095,12,1,23641 7TH AVE S 98198,23641,,,7TH,AVE,...,0,0,1929,1948,0,0,0,4,0,5119400095
295004,511940,0095,7,1,23641 7TH AVE S 98198,23641,,,7TH,AVE,...,1,0,1928,1979,0,0,0,3,3130,5119400095
295005,511940,0095,9,1,23641 7TH AVE S 98198,23641,,,7TH,AVE,...,0,0,1929,1998,0,0,0,3,3000,5119400095
295007,511940,0095,10,1,23641 7TH AVE S 98198,23641,,,7TH,AVE,...,0,0,1929,1948,0,0,0,4,0,5119400095
295008,511940,0095,3,1,23641 7TH AVE S 98198,23641,,,7TH,AVE,...,0,0,1931,2016,0,2,0,3,0,5119400095
295010,511940,0095,11,1,23641 7TH AVE S 98198,23641,,,7TH,AVE,...,0,0,1929,1973,0,0,0,4,0,5119400095


In [72]:
# remove duplicates
renovated_df = renovated_df.drop_duplicates("ParcelId", keep = False)
renovated_df.shape

(22945, 51)

In [73]:
renovated_df.ParcelId.value_counts().sort_values()

5366200195    1
4060000400    1
2122069086    1
4310702015    1
2832100670    1
7883608646    1
8131700220    1
0723049314    1
2523039158    1
3625059064    1
7168200075    1
4006000271    1
2522069021    1
7011200865    1
7579200560    1
2206500435    1
2883201380    1
2223059041    1
5216200200    1
9274200990    1
9274202595    1
7518504280    1
9274201820    1
2998300135    1
9406300045    1
8820902500    1
2815600300    1
0003600057    1
1778300280    1
4073200420    1
             ..
1566100540    1
0824059113    1
9521101183    1
1732801496    1
9274204440    1
1245002220    1
1260200005    1
5119400030    1
7173700660    1
4217402210    1
4045500225    1
1924059177    1
6052402155    1
7645900230    1
3353404520    1
7935000025    1
1523049118    1
3879901505    1
1535202445    1
1489300125    1
1330300476    1
0669000120    1
2461900240    1
0723069016    1
1326039034    1
6141100315    1
0269001070    1
5553300015    1
7203600575    1
6338000400    1
Name: ParcelId, Length: 

In [74]:
# export renovted_df
renovated_df.to_csv("Renovated.csv", index = False)

In [75]:
# II. import sale data
sale_dtype = {"ExciseTaxNbr": str, "Major": str, "Minor": str}
sale_df = pd.read_csv("EXTR_RPSale.csv", dtype = sale_dtype)
sale_df.shape

(1957142, 24)

In [76]:
sale_df.head()

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
0,714942,284150,0010,04/06/1983,91500,198304110267,100,057,284150,P,...,2,6,2,N,N,N,,1,8,
1,1729614,172204,9157,12/21/1999,0,19991229001498,,,,,...,3,11,15,N,N,N,N,10,8,31 45
2,1729614,172204,9005,12/21/1999,0,19991229001498,,,,,...,3,11,15,N,N,N,N,10,8,31 45
3,2254430,192304,9020,12/05/2006,0,20061207002200,,,,,...,3,11,15,N,N,N,N,18,2,18 45
4,685277,885730,0120,08/11/1982,0,198208170380,086,075,885730,P,...,3,2,15,N,N,N,,1,3,11


In [77]:
sale_df.iloc[1]

ExciseTaxNbr                                                    1729614
Major                                                            172204
Minor                                                              9157
DocumentDate                                                 12/21/1999
SalePrice                                                             0
RecordingNbr                                             19991229001498
Volume                                                                 
Page                                                                   
PlatNbr                                                                
PlatType                                                               
PlatLot                                                                
PlatBlock                                                              
SellerName            GRIFFIN WILLIAM R                             ...
BuyerName             DESIMONE TERESA J                         

In [78]:
# check missing values
na = (sale_df.isnull().sum() / len(sale_df)) * 100
na = na.drop(na[na == 0].index).sort_values(ascending=False)
na_df = pd.DataFrame({'Missing Ratio' :na})
na_df

,Missing Ratio


In [79]:
sale_df["ParcelId"] = sale_df["Major"]+sale_df["Minor"]
# keep only those with sale price above zero
saleAboveZero_df = sale_df[sale_df.SalePrice > 0].copy()
saleAboveZero_df.shape

(1277528, 25)

In [80]:
# keep only those in "renovated_df"
renovatedFlag = saleAboveZero_df.ParcelId.isin(renovated_df.ParcelId)
saleAndRenovated_df = saleAboveZero_df[renovatedFlag].copy()
saleAndRenovated_df.shape

(42946, 25)

In [81]:
# keep only ParcelIds that appear more than once
multiFlag = saleAndRenovated_df.duplicated("ParcelId", keep=False)
multiSale_df = saleAndRenovated_df[multiFlag].copy()
multiSale_df.shape

(35654, 25)

In [82]:
# export
multiSale_df.to_csv("MultiSale.csv", index = False)

## EDA
1. Group by parcelId, 得到time delta
2. get annual growth rate(CAGR): (EndSalePrice/BeginningSalePrice)^(365/deltaD)
3. flag - 标记并保留所有detlaT小于731的instance
4. flag - filter out annual growth rate > 1.5
5. get some descriptive info about the filtered result

In [83]:
import datetime
multiSale_df["DocumentDate"] = pd.to_datetime(multiSale_df["DocumentDate"])
multiSale_df = multiSale_df.sort_values(by = ["ParcelId", "DocumentDate"])
multiSale_df.head(10)

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,ParcelId
1893798,1397723,000100,0035,1994-09-28,850,199409301917,905,005,000100,D,...,6,2,N,N,N,N,18,8,24,0001000035
631479,1533695,000100,0035,1997-03-18,225000,199703211326,905,005,000100,D,...,6,3,N,N,N,N,1,8,51,0001000035
1900802,1867646,000100,0035,2002-02-07,195000,20020213001235,,,,,...,6,3,N,N,N,N,1,8,,0001000035
1460841,1440375,000120,0020,1995-07-24,174000,199507310897,033,001,611750,P,...,6,3,N,N,N,N,1,8,,0001200020
226855,1558666,000120,0020,1997-07-22,200000,199707312284,905,006,000120,D,...,6,3,N,N,N,N,1,8,,0001200020
1565490,1342888,000180,0010,1993-11-09,118000,199311181664,905,009,000180,D,...,0,2,N,N,N,N,1,8,,0001800010
231293,2588709,000180,0010,2013-02-06,289000,20130212001306,,,,,...,6,3,,,,,1,8,46 60,0001800010
576953,1446963,000200,0016,1995-09-05,253000,199509081019,000,000,000000,,...,6,3,N,N,N,N,1,8,51,0002000016
278575,2780069,000200,0016,2016-02-08,335500,20160212001372,,,,,...,6,18,N,N,N,N,4,8,13 31 46,0002000016
1193086,2842023,000200,0016,2016-12-28,599950,20161229001052,,,,,...,6,3,,,,,1,8,,0002000016


In [84]:
multiSaleSorted_df.DocumentDate.iloc[1] - multiSaleSorted_df.DocumentDate.iloc[0] < datetime.timedelta(days = 731)

False

In [85]:
# get time interval
multiSale_df['deltaT'] = multiSale_df.sort_values(["ParcelId", "DocumentDate"]).groupby('ParcelId')['DocumentDate'].diff()
multiSale_df.head(10)

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,ParcelId,deltaT
1893798,1397723,000100,0035,1994-09-28,850,199409301917,905,005,000100,D,...,2,N,N,N,N,18,8,24,0001000035,NaT
631479,1533695,000100,0035,1997-03-18,225000,199703211326,905,005,000100,D,...,3,N,N,N,N,1,8,51,0001000035,902 days
1900802,1867646,000100,0035,2002-02-07,195000,20020213001235,,,,,...,3,N,N,N,N,1,8,,0001000035,1787 days
1460841,1440375,000120,0020,1995-07-24,174000,199507310897,033,001,611750,P,...,3,N,N,N,N,1,8,,0001200020,NaT
226855,1558666,000120,0020,1997-07-22,200000,199707312284,905,006,000120,D,...,3,N,N,N,N,1,8,,0001200020,729 days
1565490,1342888,000180,0010,1993-11-09,118000,199311181664,905,009,000180,D,...,2,N,N,N,N,1,8,,0001800010,NaT
231293,2588709,000180,0010,2013-02-06,289000,20130212001306,,,,,...,3,,,,,1,8,46 60,0001800010,7029 days
576953,1446963,000200,0016,1995-09-05,253000,199509081019,000,000,000000,,...,3,N,N,N,N,1,8,51,0002000016,NaT
278575,2780069,000200,0016,2016-02-08,335500,20160212001372,,,,,...,18,N,N,N,N,4,8,13 31 46,0002000016,7461 days
1193086,2842023,000200,0016,2016-12-28,599950,20161229001052,,,,,...,3,,,,,1,8,,0002000016,324 days


In [92]:
# get salePrice interval
multiSale_df['deltaSalePrice'] = multiSale_df.sort_values(["ParcelId", "DocumentDate"]).groupby('ParcelId')['SalePrice'].diff()
multiSale_df.head(10)

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,ParcelId,deltaT,deltaDay,deltaSalePrice
1893798,1397723,000100,0035,1994-09-28,850,199409301917,905,005,000100,D,...,N,N,N,18,8,24,0001000035,NaT,NaN,NaN
631479,1533695,000100,0035,1997-03-18,225000,199703211326,905,005,000100,D,...,N,N,N,1,8,51,0001000035,902 days,902.0,224150.0
1900802,1867646,000100,0035,2002-02-07,195000,20020213001235,,,,,...,N,N,N,1,8,,0001000035,1787 days,1787.0,-30000.0
1460841,1440375,000120,0020,1995-07-24,174000,199507310897,033,001,611750,P,...,N,N,N,1,8,,0001200020,NaT,NaN,NaN
226855,1558666,000120,0020,1997-07-22,200000,199707312284,905,006,000120,D,...,N,N,N,1,8,,0001200020,729 days,729.0,26000.0
1565490,1342888,000180,0010,1993-11-09,118000,199311181664,905,009,000180,D,...,N,N,N,1,8,,0001800010,NaT,NaN,NaN
231293,2588709,000180,0010,2013-02-06,289000,20130212001306,,,,,...,,,,1,8,46 60,0001800010,7029 days,7029.0,171000.0
576953,1446963,000200,0016,1995-09-05,253000,199509081019,000,000,000000,,...,N,N,N,1,8,51,0002000016,NaT,NaN,NaN
278575,2780069,000200,0016,2016-02-08,335500,20160212001372,,,,,...,N,N,N,4,8,13 31 46,0002000016,7461 days,7461.0,82500.0
1193086,2842023,000200,0016,2016-12-28,599950,20161229001052,,,,,...,,,,1,8,,0002000016,324 days,324.0,264450.0


In [95]:
# get annualized growth rate
multiSale_df['deltaDay'] = multiSale_df['deltaT'].dt.days
multiSale_df.deltaDay.dtypes
multiSale_df['CAGR'] = (multiSale_df['SalePrice']/(multiSale_df['SalePrice']-multiSale_df['deltaSalePrice'])) ** (365/multiSale_df['deltaDay']) - 1
multiSale_df.head(10)

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning,ParcelId,deltaT,deltaDay,deltaSalePrice,CAGR
1893798,1397723,000100,0035,1994-09-28,850,199409301917,905,005,000100,D,...,N,N,18,8,24,0001000035,NaT,NaN,NaN,NaN
631479,1533695,000100,0035,1997-03-18,225000,199703211326,905,005,000100,D,...,N,N,1,8,51,0001000035,902 days,902.0,224150.0,8.558431
1900802,1867646,000100,0035,2002-02-07,195000,20020213001235,,,,,...,N,N,1,8,,0001000035,1787 days,1787.0,-30000.0,-0.028806
1460841,1440375,000120,0020,1995-07-24,174000,199507310897,033,001,611750,P,...,N,N,1,8,,0001200020,NaT,NaN,NaN,NaN
226855,1558666,000120,0020,1997-07-22,200000,199707312284,905,006,000120,D,...,N,N,1,8,,0001200020,729 days,729.0,26000.0,0.072215
1565490,1342888,000180,0010,1993-11-09,118000,199311181664,905,009,000180,D,...,N,N,1,8,,0001800010,NaT,NaN,NaN,NaN
231293,2588709,000180,0010,2013-02-06,289000,20130212001306,,,,,...,,,1,8,46 60,0001800010,7029 days,7029.0,171000.0,0.047613
576953,1446963,000200,0016,1995-09-05,253000,199509081019,000,000,000000,,...,N,N,1,8,51,0002000016,NaT,NaN,NaN,NaN
278575,2780069,000200,0016,2016-02-08,335500,20160212001372,,,,,...,N,N,4,8,13 31 46,0002000016,7461 days,7461.0,82500.0,0.013903
1193086,2842023,000200,0016,2016-12-28,599950,20161229001052,,,,,...,,,1,8,,0002000016,324 days,324.0,264450.0,0.924708


In [96]:
# flag: keep rows with sale time interval smaller than 731 and annual growth rate above 50%
highProfit_df = multiSale_df[(multiSale_df.deltaDay < 731) & (multiSale_df.CAGR > 0.5)].copy()
highProfit_df.shape

(2423, 29)